<img src="images/escudo_utfsm.gif" style="float:right;height:100px">
<img src="images/IsotipoDIisocolor.png" style="float:left;height:100px">
<center>
    <h1> ILI286 Computación Científica I</h1>
    <h1> Tarea N°3: SVD</h1>
    <h3> [S]cientific [C]omputing [T]eam 2018</h3>
</center>

<p>
<center>_Abril 2018_ </center>
</p>

In [1]:
import numpy as np
import IPython.display as ipd
from scipy.io.wavfile import read 

# Introducción

La descomposición de valores singulares, también conocida como SVD (Singular-Value Decomposition) , es una factorización que tiene toda matriz real o compleja. Esta herramienta, es utilizada en muchos campos tales como Machine Learning, Data visualization, Data compression, etc. En esta tarea utilizaremos un algoritmo llamado Principal Component Analysis (PCA), que como base utiliza la descomposición SVD, es por esto que se le adjunta un documento que usted deberá leer para comprender que hace la SVD y la relación que tiene con PCA. 

El objetivo de la tarea será comprimir un archivo de audio, se utilizara un extracto de la canción "Roundabout" del grupo "Yes"[[1]](#bib) publicada en 1971, dado que la tarea tiene fines academicos, nos es posible utilizarla [[2]](#bib) [[3]](#bib). Para poder extraer la data del .wav y poder escuchar la canción en su jupyter notebook puede utilizar el siguiente código.

In [2]:
#sr: Sample_Rate
#data: array 2-D (left and right channel)
sr, data = read("tbc.wav")
ipd.Audio([data[:,0],data[:,1]], rate=sr)
sr

44100

# Pregunta 1 (35 pts)

Como usted bien habra leido, para poder usar la SVD es necesario tener una representación matricial de la canción, esto se puede lograr cortando en trozos iguales la canción y apilarlos como vectores fila. Para el resto de la tarea nos referiremos a los trozos de la canción como ventanas.

<figure>
  <img src="images/example3.png" height="28%" width = "28%" >
  <center>
  <figcaption>Figura 1: Representación de ambos canales</figcaption>
  </center>
</figure>

Para contextualizar mejor, usted podría pensar que su matriz de audio es equivalente a la matriz $W$(7) de la sección Principal Components Analysis del texto adjunto. Luego de aplicar SVD a la matriz W usted deberá encontrar los coeficientes $w_{i}$, de tal manera que al multiplicar los coeficientes con las bases que usted obtuvo mediante SVD, podrá recuperar $W$. Es importante notar que usted no necesariamente tiene que tomar todas las bases, si no que perfectamente podría tomar las que aporten una mayor información de la matriz $W$ (ver el concepto de varianza), todas las bases encontradas que ayudan a reconstruir $W$ son conocidas componentes principales.

a) Genere una función que realice el algoritmo PCA utilizando SVD (numpy.linalg.svd), esta función debe retornar el diccionario new_format que contendrá información relevante para la reconstrucción y reproducción de la data. Si el flag Save_pickle es verdadero la función además deberá crear un archivo binario .pickle [[4]](#bib) donde en su interior será almacenado el diccionario new_format.

```python
'''
Data - ((MxN)-Array) Matrix with the partitions of the song (Left and Right Channel)
D - (integer) Number of principal components.
Save_pickle - (Boolean) if save_pickle it's True, this function must save a pickle
'''
   
def generate_new_format(Data, D, save_pickle=True):
    ...
    
    '''
    ############## Dictionary Content ##############
    Sample_Rate - (Integer) Number of samples of audio carried per second 
    Windows_size - (Integer) Number of samples in a partition.
    Number_of_partitions - (Integer) Number of partitions. 
    Principal_components - ((DxN)-Array) The D principal components of the matrix Data.
    Coeficients_for_reconstruction- ((NxD)-Array) The D Coeficients of PCA. 
    '''
    new_format = {}
    new_format['Sample_Rate'] = ...
    new_format['Windows_size'] = ..
    new_format['Number_of_partitions'] = 
    new_format['Principal_components'] = 
    new_format['Coeficients_for_reconstruction'] = 
    
    ....
    
    return(new_format)
```  

In [ ]:
new_format['Principal_components'] = np.dot(u,smat)[:,:D]
new_format['Coeficients_for_reconstruction'] = vh[:D,:]

In [25]:
import numpy as np
from scipy.io.wavfile import write

def matrix_generator(Data,Ventana):
    l_channel = np.matrix(np.split(Data[:,0].flatten(),Ventana))
    r_channel = np.matrix(np.split(Data[:,1].flatten(),Ventana))
    final_matrix = np.concatenate((l_channel, r_channel), axis=0)
    print(final_matrix.shape)
    return final_matrix

def generate_new_format(Data, D, Ventana,sr,save_pickle=True):
    matrix = matrix_generator(Data,Ventana)
    media = matrix.mean(0)
    Z = np.subtract(matrix, media)
    u, s, vh = np.linalg.svd(Z, full_matrices=False)
    print(u.shape,s.shape,vh.T.shape,Z.shape)
    Y = np.dot(Z,vh.T)
    Y2 = np.dot(u,np.diag(s))
    arrayNxD = Y2[:,:D]
    arrayDxN = vh[:D,:]
    #C = np.linalg.solve(Y,Z)
    #Principal_components = Y2[0:D,:]
    #Coeficients_for_reconstruction = C[0:D,:]
    #print("Componentes:")
    #print(Coeficients_for_reconstruction.shape,Principal_components.shape)
    #aprox_z  = np.dot(Coeficients_for_reconstruction.T,Principal_components)
    #print(aprox_z.shape)
    
    new_format = {}
    new_format['Sample_Rate'] = sr
    new_format['Windows_size'] = Ventana
    new_format['Number_of_partitions'] = Z.shape[0]
    new_format['Principal_components'] = arrayDxN
    new_format['Coeficients_for_reconstruction'] = arrayNxD
    new_format['mean'] = media
    
    return new_format
    
    
new_format = generate_new_format(data,1024,328,sr)

Z = np.dot(new_format['Coeficients_for_reconstruction'],new_format['Principal_components'])
W = Z + new_format['mean']
L_matrix = np.split(W,2,axis=0)[0]
R_matrix = np.split(W,2,axis=0)[1]
size = int(new_format['Windows_size']*new_format['Number_of_partitions']/2)
L_matrix = np.reshape(L_matrix,(size,1))
R_matrix = np.reshape(R_matrix,(size,1))
new_data = [np.asarray(L_matrix).reshape(-1),np.asarray(R_matrix).reshape(-1)]
ipd.Audio(new_data,rate=new_format['Sample_Rate'])

(656, 7808)
(656, 656) (656,) (7808, 656) (656, 7808)


ValueError: cannot reshape array of size 2561024 into shape (107584,1)

In [12]:
import numpy
sr = 22050 # sample rate
T = 2.0    # seconds
t = numpy.linspace(0, T, int(T*sr), endpoint=False) # time variable
x = 0.5*numpy.sin(2*numpy.pi*440*t)                # pure sine wave at 440 Hz
print(x)

[ 0.          0.06252526  0.12406892 ... -0.1836648  -0.12406892
 -0.06252526]


## Pregunta 2 (25 pts)

a) Genere un widget que reproduzca la aproximación del audio y ademas genere dos gráficos a medida que se hace variar el nº de componentes principales y tamaño de la ventana. El primer gráfico debe mostrarse la señal de audio original con la señal aproximada. El segundo gráfico debe ser un histograma del logaritmo del error absoluto. Concluya. (Realice para ambos canales de audio) (Hint: El numero maximo de componentes principales varía según el tamaño de la ventana). 
<figure>
  <img src="images/example.png" height="68%" width = "68%" >
  <center>
  <figcaption>Figura 2: Ejemplo de graficos utilizando una componente principal</figcaption>
  </center>
</figure>

b) Genere un widget que haga cambiar el tamaño de la ventana, este debe mostrar en un solo gráfico el comportamiento del error absoluto y el cuociente entre el .wav con el .pickle a medida que cambia la cantidad de componentes principales.
(Guarde las series de tiempo y sample rate del .wav en un pickle para comparar los archivos)

# Pregunta 3 (30 pts)

Genere una función que reciba como parámetro el nombre de un archivo .wav y el porcentaje de reducción del tamaño del archivo. 
Esta función tendrá que encontrar la combinación del tamaño de ventana y número de componentes principales  tal que se minimice la norma $l_{2}$ entre la señal original con su aproximación, y reduzca el tamaño del archivo original al porcentaje de entrada _reduction_percentage_. 
Se debe comparar el tamaño de los archivos de igual manera que la pregunta 2.

¿Es la norma $l_{2}$ una buena métrica para medir la calidad de un audio? 

¿Existen alguna mejor métrica para realizar la comparación? Concluya.
```python
'''
filename - (String) Name of the input file.
reduction_percentage - (float) Corresponds to the percentage that you want to reduce the file size
'''  
def compress_file(filename, reduction_percentage):
    ...
```  

# Instrucciones:

* La estructura del laboratorio es la siguiente:
     1. Título, nombre de estudiante, email y rol.
     2. Introducción.
     3. Desarrollo y análisis de resultados.
     4. Conclusiones.
     5. Referencias.
* El laboratorio debe ser realizado en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, `SymPy`, `Matplotlib` y `ipywidgets`, así como la correcta implementación de algoritmos vectorizados.
* El archivo de entrega debe denominarse Tarea3-rol.tar.gz y debe contener un directorio con todos los archivos necesarios para ejecutar el notebook, junto con un archivo README indicando explícitamente las librerías o módulos utilizados, nombre y rol del estudiante. El _notebook_ debe tener como nombre Tarea2-rol.ipynb.
* El descuento por día de atraso será de $30$ puntos, con un máximo de 1 día de atraso. No se recibirán entregas después de este día.
* Debe citar toda fuente de código externo. 
* El trabajo es personal, no se permite compartir código ni utilizar código de otros, aunque sí se sugiere discutir aspectos generales con sus compañeros.
* En caso de sospecha de no cumplimiento de estas instrucciones, se solicitará al involucrado o la involucrada a aclarar la situación. Dependiendo de la justificación se decidirá su calificación, la cual podrá o no ser penalizada.
* El no seguir estas instrucciones, implica descuentos en su nota obtenida.

<a id='bib'></a>
## Referencias

* <a href=http://www.yesworld.com>[1] Yes Official Website </a>
* <a href=http://www.copyright.com>[2] Copyright </a>
* <a href=https://www.bcn.cl/leyfacil/recurso/propiedad-intelectual-(derechos-de-autor)>[3] Derechos de autor, Biblioteca del congreso Nacional de Chile</a>
* <a href=https://docs.python.org/3/library/pickle.html>[4] Documentación Pickle